***Step 1: Importing the Libraries***

In [5]:
import gensim
import pandas as pd
import numpy as np
import nltk
import string

In [ ]:
# Now lets import the Data Frame and look at the contents of it

***Step 2: Importing the Data Frame***

In [45]:
dir="C:\\NLP in Python\\Topic Model\\"
df=pd.read_csv(dir+'fy2015_standard_inventory_report.csv')
df.head()

,Product or Service Code,Product or Service Description,Contracting Agency Name,Contracting Department Name,Funding Agency Name,Principal Place of Performance City Name,Principal Place of Performance State Code,Principal Place of Performance Country Name,Signed Date,Extent Competed,Fair Opportunity/Limited Sources,Type of Contract,Description of Requirement,Vendor Name,PIID,Referenced IDV PIID,DUNS Number,Action Obligation
0,AB92,R&D- COMMUNITY SERVICE/DEVELOPMENT: OTHER (APP...,ENVIRONMENTAL PROTECTION AGENCY,ENVIRONMENTAL PROTECTION AGENCY,ENVIRONMENTAL PROTECTION AGENCY,NaN,NaN,NaN,11/3/2014,FULL AND OPEN COMPETITION,NaN,COST PLUS FIXED FEE,CLOSELY ASSOCIATED - ENVIRONMENTAL INDICATORS ...,"EASTERN RESEARCH GROUP, INC.",EPC12028,NaN,112947395,"$131,940.00"
1,AB92,R&D- COMMUNITY SERVICE/DEVELOPMENT: OTHER (APP...,ENVIRONMENTAL PROTECTION AGENCY,ENVIRONMENTAL PROTECTION AGENCY,ENVIRONMENTAL PROTECTION AGENCY,NaN,NaN,NaN,12/24/2014,FULL AND OPEN COMPETITION,NaN,COST PLUS FIXED FEE,CLOSELY ASSOCIATED - ENVIRONMENTAL INDICATORS ...,"EASTERN RESEARCH GROUP, INC.",EPC12028,NaN,112947395,"$113,846.00"
2,AB92,R&D- COMMUNITY SERVICE/DEVELOPMENT: OTHER (APP...,ENVIRONMENTAL PROTECTION AGENCY,ENVIRONMENTAL PROTECTION AGENCY,ENVIRONMENTAL PROTECTION AGENCY,NaN,NaN,NaN,3/3/2015,FULL AND OPEN COMPETITION,NaN,COST PLUS FIXED FEE,CLOSELY ASSOCIATED - ENVIRONMENTAL INDICATORS ...,"EASTERN RESEARCH GROUP, INC.",EPC12028,NaN,112947395,"$62,055.00"
3,AB92,R&D- COMMUNITY SERVICE/DEVELOPMENT: OTHER (APP...,ENVIRONMENTAL PROTECTION AGENCY,ENVIRONMENTAL PROTECTION AGENCY,ENVIRONMENTAL PROTECTION AGENCY,NaN,NaN,NaN,4/7/2015,FULL AND OPEN COMPETITION,NaN,COST PLUS FIXED FEE,CLOSELY ASSOCIATED - ENVIRONMENTAL INDICATORS ...,"EASTERN RESEARCH GROUP, INC.",EPC12028,NaN,112947395,"$58,714.00"
4,AB92,R&D- COMMUNITY SERVICE/DEVELOPMENT: OTHER (APP...,ENVIRONMENTAL PROTECTION AGENCY,ENVIRONMENTAL PROTECTION AGENCY,ENVIRONMENTAL PROTECTION AGENCY,NaN,NaN,NaN,4/30/2015,FULL AND OPEN COMPETITION,NaN,COST PLUS FIXED FEE,CLOSELY ASSOCIATED - ENVIRONMENTAL INDICATORS ...,"EASTERN RESEARCH GROUP, INC.",EPC12028,NaN,112947395,"$150,000.00"


In [29]:
# Number of records
df.shape

# Number of records is 5150

(5149, 19)

In [99]:
# After examining the columns and values, the following things can be said:
# 1: It is some kind of purchase sheet (deduced from PIID column)
# 2: PIID stands for Procurement Instrument Identifier
# 3: Vendor Name: The party from which the purchase has been made
# 4: Signed Data: Date on which the agreement has been signed
# 5: Action Obligation: Contains contract price information

# The Aim of the activity is to use 'Description of Requirement' column and extract meaningful insights from it
# We will try to assign each row of the column to a particular high level construct/Topic

***Step 3: Creating Custom Stop Word List***

In [375]:
# We need to also create the stop word list so that we can use it here to remove the stop words

from nltk.corpus import stopwords
stop = set(stopwords.words('english'))
stop2=list(stop)
stop2.extend(['IGF::OT::IGF','AND','FOR','OF','THE','TO','IN','THIS','IS','IGF::CL::IGF',
             'IGF::CT::IGF','AT','A','NEW','AIR','WITH','OTHER','ADD','UNDER','FROM','1','2',
             'AS','ON','NO.','IV','IGF::CLCT::IGF','BY','TOTAL','IT','II'])
stop2.extend(stop_word_2)
stop2[len(stop2)-1]

'WATER  TREATMENT  SITE'

***Step 4: Defining Custom Functions***

In [376]:
# Before we assign Topics to each row, it is necessary to look at the distribution of words
# For doing Topic Modelling, we need to create Features which requires us to analyse ngrams
# Let us use the functions defined in the previous blogs

# White space tokenizer
def tokenize(text):
    tokenizer = nltk.tokenize.WhitespaceTokenizer()
    token = []
    for item in text:
        token.append(tokenizer.tokenize(item))
        
    # Removing stop words
    ls_dummy1=[]
    ls_dummy2=[]
    temp=1
    for i in token:
        ls_dummy1=[]
        for j in i:
            if j in stop2:
                temp=1
            else:
                ls_dummy1.append(j)
        ls_dummy2.append(ls_dummy1)
    l3=ls_dummy2
        
    return l3

# Remove Punctuations
# Creating a function for removing the puntuation from text
def rem_punctuation(data_frame,colname):
    l3=[]
    l2=[i for i in data_frame[colname]]
    l3=tokenize(l2)

    # Removing the punctuations
    l5=[]
    l6=[]
    for j in l3:
        for k in j:
            if k in string.punctuation:
                temp=1
            else:
                l5.append(k)
        l6.append(l5)
        l5=[]
    c_ls=[" ".join(i) for i in l6]
    df1=pd.DataFrame(c_ls)
    df1.columns=['Text']
    return(df1)

# We will now create a function that takes in text and ngram list
# text/document is stores in a list
# list also specifies ngram paramter
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import word_tokenize

# Create Ngrams distribution upto n=4
# The function should take the column containing text data
def TF_NGRAM(data_frame,colname,ngram):
    # Tokenize the data
    DocID_ls=[]

    #Appending the Doc_ID column
    data_frame['Doc_ID']=[i for i in range(1,data_frame[colname].shape[0]+1)]


    from nltk import ngrams       
    l1=[]
    for i in data_frame['Doc_ID']:
        for k in data_frame[data_frame['Doc_ID']==i][colname]:
            for j in ngram:
                unigrams = ngrams(k.split(), j)
                for z in unigrams:
                    if j==1:
                        l1.append(z[0])
                    elif j==2:
                        l1.append(z[0] + " , " +z[1])
                    elif j==3:
                        l1.append(z[0] + " , " +z[1]+" , " +z[2])
                    elif j==4:
                        l1.append(z[0] + " , " +z[1]+" , " +z[2] +" , " +z[3])

                    DocID_ls.append(i)

    # Creating a Data Frame out of it

    s1=pd.Series(l1)
    s2=pd.Series(DocID_ls)
    df1=pd.DataFrame(s2)
    df1['s1']=s1
    df1.columns=['Doc_ID','Tokens']
    df1
    return(df1)


***Step 5: Removing Punctuations from the Text***

In [377]:
# Before we start, we need to remove the \n identifier in the cell by ""
# Removing punctuations, stop words etc
pd.set_option('display.max_colwidth', -1)
df1=rem_punctuation(df,'Description of Requirement')
df1.head(20)


,Text
0,"CLOSELY ASSOCIATED ENVIRONMENTAL PROGRAM SCIENTIFIC, TECHNICAL, SUPPORT SERVICES"
1,"CLOSELY ASSOCIATED ENVIRONMENTAL PROGRAM SCIENTIFIC, TECHNICAL, SUPPORT SERVICES"
2,"CLOSELY ASSOCIATED ENVIRONMENTAL PROGRAM SCIENTIFIC, TECHNICAL, SUPPORT SERVICES"
3,"CLOSELY ASSOCIATED ENVIRONMENTAL PROGRAM SCIENTIFIC, TECHNICAL, SUPPORT SERVICES"
4,"CLOSELY ASSOCIATED ENVIRONMENTAL PROGRAM SCIENTIFIC, TECHNICAL, SUPPORT SERVICES"
5,
6,CONTRACTOR PROVIDE SERVICES EPA SUPPORT
7,SUPERFUND MANAGEMENT OFFICE SUPPORT FUNDING ACTION
8,"MAINTENANCE CINCINNATI,"
9,SUPPORT SERVICE EPA


***Step 6: Creating the N-Gram Frequency Distribution***

In [378]:
# ngram frequency analysis
# checking the ngram distribution
ngram_df=TF_NGRAM(df1,'Text',[1,2,3])

In [379]:
# Check the first few records
ngram_df.head(20)

,Doc_ID,Tokens
0,1,CLOSELY
1,1,ASSOCIATED
2,1,ENVIRONMENTAL
3,1,PROGRAM
4,1,"SCIENTIFIC,"
5,1,"TECHNICAL,"
6,1,SUPPORT
7,1,SERVICES
8,1,"CLOSELY , ASSOCIATED"
9,1,"ASSOCIATED , ENVIRONMENTAL"


In [380]:
 # checking the number of unique 'Doc_ID' in ngram_df
len(ngram_df['Doc_ID'].unique()) 
# We can see that the number of unique Doc_ID is less than 5149

4601

In [381]:
# Replace all , in Tokens with ""
import re
ngram_df['Tokens']=ngram_df['Tokens'].apply(lambda x: re.sub(',','',x))
ngram_df.head(20)


,Doc_ID,Tokens
0,1,CLOSELY
1,1,ASSOCIATED
2,1,ENVIRONMENTAL
3,1,PROGRAM
4,1,SCIENTIFIC
5,1,TECHNICAL
6,1,SUPPORT
7,1,SERVICES
8,1,CLOSELY ASSOCIATED
9,1,ASSOCIATED ENVIRONMENTAL


In [382]:
# checking the number of unique 'Doc_ID' in ngram_df
len(ngram_df['Doc_ID'].unique())

4601

***Step 7: Creating the Frequency Profile***

In [383]:
# Creating a summary at Doc_ID and Tokens level
ngram_df['Frequency']=1
ngram_df0=ngram_df.groupby(['Doc_ID','Tokens'])['Frequency'].sum().reset_index()
ngram_df0.head()

,Doc_ID,Tokens,Frequency
0,1,ASSOCIATED,1
1,1,ASSOCIATED ENVIRONMENTAL,1
2,1,ASSOCIATED ENVIRONMENTAL PROGRAM,1
3,1,CLOSELY,1
4,1,CLOSELY ASSOCIATED,1


In [385]:
# Creating the summary at an overall level
freq_dist=nltk.FreqDist(ngram_df0['Tokens'])
df5=pd.DataFrame(pd.Series(freq_dist),columns=['Count'])
df5.sort_values(['Count'])
df6=df5.sort_values(['Count'],ascending=False).reset_index()
df6.shape # 827k records
df6.head(20)

,index,Count
0,SUPPORT,1566
1,SERVICES,1067
2,ORDER,970
3,TASK,907
4,TASK ORDER,841
5,FUNDING,767
6,MODIFICATION,678
7,CONTRACT,604
8,INCREMENTAL,456
9,TECHNICAL,429


In [386]:
df6.shape

(13079, 2)

In [388]:
# Writing the data frame to drive
df6.to_csv(dir + "\\ngrams.csv")

In [359]:
# Identifying another set of stop word list 
# These are the words which have frequency count less than 100
stop_word_2=list(df6['index'][df6['Count']< 100])
len(stop_word_2)

61046

***Step 8:Selecting Features***

In [389]:
# df6 contains the unigrams, bigrams and trigrams frequency distribution
# We are now going to use those words from df6 which have Count value more than 100
# This is based on the assumption that words used most frequently are able to describe a document better
Features=list(df6['index'][df6['Count']> 100])
Features[1:10]


['SERVICES',
 'ORDER',
 'TASK',
 'TASK  ORDER',
 'FUNDING',
 'MODIFICATION',
 'CONTRACT',
 'INCREMENTAL',
 'TECHNICAL']

In [390]:
len(Features)

88

In [ ]:
# As we can see from the Frequency Distribution of words that there are a lot of words like
# IGF::OT::IGF,AND, FORm THE etc that done impart any meaning to the text corpus
# So we need to remove them using a custom stop word list
# If you look closely in 'tokenize' function, there is a 'stop2' list that is used to remove stop words
# We need to create the list so that we can use it here to remove the stop words
# The list has been created at Step 3. The list is updated by analysing the result of Step 7


***Step 9: Creation the Dictionary and  Document Term Matrix(DTM)***

In [401]:
ls=[]
for i in ngram_df0['Doc_ID'].unique():
    ls.append(list(ngram_df0['Tokens'][ngram_df0['Doc_ID']==i]))

In [402]:
ls[0:2]

[['ASSOCIATED',
  'ASSOCIATED  ENVIRONMENTAL',
  'ASSOCIATED  ENVIRONMENTAL  PROGRAM',
  'CLOSELY',
  'CLOSELY  ASSOCIATED',
  'CLOSELY  ASSOCIATED  ENVIRONMENTAL',
  'ENVIRONMENTAL',
  'ENVIRONMENTAL  PROGRAM',
  'ENVIRONMENTAL  PROGRAM  SCIENTIFIC',
  'PROGRAM',
  'PROGRAM  SCIENTIFIC',
  'PROGRAM  SCIENTIFIC  TECHNICAL',
  'SCIENTIFIC',
  'SCIENTIFIC  TECHNICAL',
  'SCIENTIFIC  TECHNICAL  SUPPORT',
  'SERVICES',
  'SUPPORT',
  'SUPPORT  SERVICES',
  'TECHNICAL',
  'TECHNICAL  SUPPORT',
  'TECHNICAL  SUPPORT  SERVICES'],
 ['ASSOCIATED',
  'ASSOCIATED  ENVIRONMENTAL',
  'ASSOCIATED  ENVIRONMENTAL  PROGRAM',
  'CLOSELY',
  'CLOSELY  ASSOCIATED',
  'CLOSELY  ASSOCIATED  ENVIRONMENTAL',
  'ENVIRONMENTAL',
  'ENVIRONMENTAL  PROGRAM',
  'ENVIRONMENTAL  PROGRAM  SCIENTIFIC',
  'PROGRAM',
  'PROGRAM  SCIENTIFIC',
  'PROGRAM  SCIENTIFIC  TECHNICAL',
  'SCIENTIFIC',
  'SCIENTIFIC  TECHNICAL',
  'SCIENTIFIC  TECHNICAL  SUPPORT',
  'SERVICES',
  'SUPPORT',
  'SUPPORT  SERVICES',
  'TECHNICAL',
 

In [394]:
# The Dictionary
from gensim import corpora
dictionary=corpora.Dictionary(ls)

In [403]:
# The Document Term Matrix
doc_term_matrix = [dictionary.doc2bow(doc) for doc in ls]
doc_term_matrix[0:3]

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (49, 1)],
 [(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (49, 1)],
 [(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (49, 1)]]

***Step 10: Hyperparamter tuning the Topic Model***

In [501]:
# Tuning for Number of Topics,alpha and eta(or sometimes also called as beta)
from gensim.test.utils import common_corpus, common_dictionary
from gensim.sklearn_api import LdaTransformer

from sklearn.model_selection import GridSearchCV
search_params = {'num_topics': [2, 3, 4, 5, 6, 7], 'alpha': [0.1, 0.2],'eta':[0.1,0.2,0.3,0.4]}
# Init the Model
lda = LdaTransformer()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)
model.get_params()

{'cv': 'warn',
 'error_score': 'raise-deprecating',
 'estimator__alpha': 'symmetric',
 'estimator__chunksize': 2000,
 'estimator__decay': 0.5,
 'estimator__dtype': numpy.float32,
 'estimator__eta': None,
 'estimator__eval_every': 10,
 'estimator__gamma_threshold': 0.001,
 'estimator__id2word': None,
 'estimator__iterations': 50,
 'estimator__minimum_probability': 0.01,
 'estimator__num_topics': 100,
 'estimator__offset': 1.0,
 'estimator__passes': 1,
 'estimator__random_state': None,
 'estimator__scorer': 'perplexity',
 'estimator__update_every': 1,
 'estimator': LdaTransformer(alpha='symmetric', chunksize=2000, decay=0.5,
         dtype=<class 'numpy.float32'>, eta=None, eval_every=10,
         gamma_threshold=0.001, id2word=None, iterations=50,
         minimum_probability=0.01, num_topics=100, offset=1.0, passes=1,
         random_state=None, scorer='perplexity', update_every=1),
 'fit_params': None,
 'iid': 'warn',
 'n_jobs': None,
 'param_grid': {'num_topics': [2, 3, 4, 5, 6, 7],


In [502]:
# Do the Grid Search
#ngram_df_DTM3=ngram_df_DTM2.head(100)
model.fit(doc_term_matrix)


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=LdaTransformer(alpha='symmetric', chunksize=2000, decay=0.5,
        dtype=<class 'numpy.float32'>, eta=None, eval_every=10,
        gamma_threshold=0.001, id2word=None, iterations=50,
        minimum_probability=0.01, num_topics=100, offset=1.0, passes=1,
        random_state=None, scorer='perplexity', update_every=1),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'num_topics': [2, 3, 4, 5, 6, 7], 'alpha': [0.1, 0.2], 'eta': [0.1, 0.2, 0.3, 0.4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [503]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)


Best Model's Params:  {'alpha': 0.1, 'eta': 0.4, 'num_topics': 2}
Best Log Likelihood Score:  -19.3028050786307


***Step 11: Training the Topic Model(LDA) with Hyper Paramters***

In [465]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel


In [504]:
# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=2,id2word = dictionary,passes=20,alpha=0.1,eta=0.4)

In [505]:
# https://github.com/priya-dwivedi/Deep-Learning/blob/master/topic_modeling/LDA_Newsgroup.ipynb?source=post_page---------------------------
# https://towardsdatascience.com/topic-modelling-in-python-with-nltk-and-gensim-4ef03213cd21
for idx, topic in ldamodel.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

Topic: 0 
Words: 0.148*"SUPPORT" + 0.126*"SERVICES" + 0.044*"EPA" + 0.043*"SUPPORT  SERVICES" + 0.037*"TECHNICAL" + 0.035*"MANAGEMENT" + 0.033*"ENVIRONMENTAL" + 0.032*"MAINTENANCE" + 0.030*"PROGRAM" + 0.026*"OFFICE"


Topic: 1 
Words: 0.073*"ORDER" + 0.065*"TASK" + 0.060*"TASK  ORDER" + 0.057*"FUNDING" + 0.054*"MODIFICATION" + 0.035*"INCREMENTAL" + 0.035*"PURPOSE" + 0.031*"PERIOD" + 0.030*"OPTION" + 0.029*"CONTRACT"




In [506]:
# Getting the topics for our corpus
print(ldamodel.get_document_topics(dictionary.doc2bow(ls[0])))

[(0, 0.990196)]


***Step 11: Creating a data frame with Topic and Top 20 Keywords in two separate columns***

In [507]:
# Get Document Topic Probability
# https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/
ldamodel.show_topics(num_topics=3, num_words=10, log=False, formatted=True)

[(0,
  '0.148*"SUPPORT" + 0.126*"SERVICES" + 0.044*"EPA" + 0.043*"SUPPORT  SERVICES" + 0.037*"TECHNICAL" + 0.035*"MANAGEMENT" + 0.033*"ENVIRONMENTAL" + 0.032*"MAINTENANCE" + 0.030*"PROGRAM" + 0.026*"OFFICE"'),
 (1,
  '0.073*"ORDER" + 0.065*"TASK" + 0.060*"TASK  ORDER" + 0.057*"FUNDING" + 0.054*"MODIFICATION" + 0.035*"INCREMENTAL" + 0.035*"PURPOSE" + 0.031*"PERIOD" + 0.030*"OPTION" + 0.029*"CONTRACT"')]

In [508]:
# Gives Keywords in each topic
for index, topic in ldamodel.show_topics(formatted=False, num_words= 30):
    print('Topic: {} \nWords: {}'.format(index, [w[0] for w in topic]))

Topic: 0 
Words: ['SUPPORT', 'SERVICES', 'EPA', 'SUPPORT  SERVICES', 'TECHNICAL', 'MANAGEMENT', 'ENVIRONMENTAL', 'MAINTENANCE', 'PROGRAM', 'OFFICE', 'REGION', 'FUNCTIONS', 'TECHNICAL  SUPPORT', 'PROVIDE', 'WATER', 'INFORMATION', 'OPERATIONS', 'NATIONAL', 'CONTRACT', 'DEVELOPMENT', 'CENTER', 'ANALYTICAL', 'SERVICE', 'ANALYSIS', 'STUDENT', 'ASSOCIATED', 'RESEARCH', 'DATA', 'CLOSELY', 'CLOSELY  ASSOCIATED']
Topic: 1 
Words: ['ORDER', 'TASK', 'TASK  ORDER', 'FUNDING', 'MODIFICATION', 'INCREMENTAL', 'PURPOSE', 'PERIOD', 'OPTION', 'CONTRACT', 'INCREMENTAL  FUNDING', 'PURPOSE  MODIFICATION', 'AMOUNT', 'OPTION  PERIOD', 'SITE', 'ALL', 'TERMS', 'FUNDING  AMOUNT', 'ACTION', 'TERMS  CONDITIONS', 'CONDITIONS', 'ALL  TERMS', 'ALL  TERMS  CONDITIONS', 'WORK', 'SUPPORT', 'FUNDS', 'REMAIN', 'EXERCISE', 'SUPERFUND', 'EXERCISE  OPTION']


In [518]:
# Storing the topic number

ls_Topic=[]
for index in ldamodel.show_topics(formatted=False, num_words= 15):
    #print(index[0])
    ls_Topic.append(index[0])
    
ls_Topic

[0, 1]

In [519]:
# Storing top keywords

ls_Keywords=[]
for topic in ldamodel.show_topics(formatted=False, num_words= 15):
    #print([w[0] for w in topic[1]])
    ls_Keywords.append([w[0] for w in topic[1]])
    
print(ls_Keywords)

[['SUPPORT', 'SERVICES', 'EPA', 'SUPPORT  SERVICES', 'TECHNICAL', 'MANAGEMENT', 'ENVIRONMENTAL', 'MAINTENANCE', 'PROGRAM', 'OFFICE', 'REGION', 'FUNCTIONS', 'TECHNICAL  SUPPORT', 'PROVIDE', 'WATER'], ['ORDER', 'TASK', 'TASK  ORDER', 'FUNDING', 'MODIFICATION', 'INCREMENTAL', 'PURPOSE', 'PERIOD', 'OPTION', 'CONTRACT', 'INCREMENTAL  FUNDING', 'PURPOSE  MODIFICATION', 'AMOUNT', 'OPTION  PERIOD', 'SITE']]


In [520]:
# Topic Number for the Data Frame
Topic_Number=[ls_Keywords.index(i) for i in ls_Keywords for j in i]
print(Topic_Number)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [521]:
# Keywords for the Data Frame
Top_Keywords=[j for i in ls_Keywords for j in i]
print(Top_Keywords)

['SUPPORT', 'SERVICES', 'EPA', 'SUPPORT  SERVICES', 'TECHNICAL', 'MANAGEMENT', 'ENVIRONMENTAL', 'MAINTENANCE', 'PROGRAM', 'OFFICE', 'REGION', 'FUNCTIONS', 'TECHNICAL  SUPPORT', 'PROVIDE', 'WATER', 'ORDER', 'TASK', 'TASK  ORDER', 'FUNDING', 'MODIFICATION', 'INCREMENTAL', 'PURPOSE', 'PERIOD', 'OPTION', 'CONTRACT', 'INCREMENTAL  FUNDING', 'PURPOSE  MODIFICATION', 'AMOUNT', 'OPTION  PERIOD', 'SITE']


In [522]:
# Combining Topic_Number and Top_Keywords into a Data Frame
Top_Keywords_df=pd.DataFrame(Topic_Number,columns=['Topic_Number'])
Top_Keywords_df['Top_Keywords']=Top_Keywords
Top_Keywords_df

,Topic_Number,Top_Keywords
0,0,SUPPORT
1,0,SERVICES
2,0,EPA
3,0,SUPPORT SERVICES
4,0,TECHNICAL
5,0,MANAGEMENT
6,0,ENVIRONMENTAL
7,0,MAINTENANCE
8,0,PROGRAM
9,0,OFFICE


***Step 12: Providing Description of the Topics based on Keywords***

In [ ]:
# If you look at the top keywords for Topic:0, most of them point to 'Support' and for Topic:1, it is about 'Contract'

***Step 13: Assigning Topics to the input text data***

In [529]:
# The dictionary from ls has been made by processing the text
# And ls has been derived from ngram_df0
# So for the Doc_IDs mentioned in ngram_df0, we can derive the Topic Assignments

ldamodel.get_document_topics(dictionary.doc2bow(ls[0]))

# for ls[0], we can see that the Topic assignment is towards Topic:0
# We will now do this for all the elements of ls

[(0, 0.990196)]

In [542]:
Topic_Number=[ldamodel.get_document_topics(dictionary.doc2bow(i))[0][0] for i in ls]
Topic_Number[0:10]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [545]:
Topic_df=pd.DataFrame(Topic_Number,columns=['Topic_Number'])
Topic_df['Topic_Number'].value_counts()

0    4365
1    236 
Name: Topic_Number, dtype: int64

In [548]:
# As you can see from the above distribution that 'Support' related documents are present more in the text corpus
# in comparison to 'Contract'
Topic_df.shape # has 4601 records
len(ngram_df0['Doc_ID'].unique())

4601

In [551]:
# Creating a Data Frame with Doc_IDs from ngram_df0 and Topic_df
Topic_df['Doc_ID']=ngram_df0['Doc_ID'].unique()
Topic_df.head(20)

,Topic_Number,Doc_ID
0,0,1
1,0,2
2,0,3
3,0,4
4,0,5
5,0,7
6,0,8
7,0,9
8,0,10
9,0,11


In [558]:
# Joining this with the main data frame df
# Adding Row_Num to it
df['Doc_ID']=list(range(1,df.shape[0]+1))
df.head()

,Product or Service Code,Product or Service Description,Contracting Agency Name,Contracting Department Name,Funding Agency Name,Principal Place of Performance City Name,Principal Place of Performance State Code,Principal Place of Performance Country Name,Signed Date,Extent Competed,Fair Opportunity/Limited Sources,Type of Contract,Description of Requirement,Vendor Name,PIID,Referenced IDV PIID,DUNS Number,Action Obligation,Doc_ID,Row_Num
0,AB92,R&D- COMMUNITY SERVICE/DEVELOPMENT: OTHER (APPLIED RESEARCH/EXPLORATORY DEVELOPMENT),ENVIRONMENTAL PROTECTION AGENCY,ENVIRONMENTAL PROTECTION AGENCY,ENVIRONMENTAL PROTECTION AGENCY,NaN,NaN,NaN,11/3/2014,FULL AND OPEN COMPETITION,NaN,COST PLUS FIXED FEE,"CLOSELY ASSOCIATED - ENVIRONMENTAL INDICATORS AND REPORT ON THE ENVIRONMENT PROGRAM (ROE) SCIENTIFIC, TECHNICAL, AND ADMINISTRATIVE SUPPORT SERVICES \nIGF::CL::IGF","EASTERN RESEARCH GROUP, INC.",EPC12028,NaN,112947395,"$131,940.00",1,1
1,AB92,R&D- COMMUNITY SERVICE/DEVELOPMENT: OTHER (APPLIED RESEARCH/EXPLORATORY DEVELOPMENT),ENVIRONMENTAL PROTECTION AGENCY,ENVIRONMENTAL PROTECTION AGENCY,ENVIRONMENTAL PROTECTION AGENCY,NaN,NaN,NaN,12/24/2014,FULL AND OPEN COMPETITION,NaN,COST PLUS FIXED FEE,"CLOSELY ASSOCIATED - ENVIRONMENTAL INDICATORS AND REPORT ON THE ENVIRONMENT PROGRAM (ROE) SCIENTIFIC, TECHNICAL, AND ADMINISTRATIVE SUPPORT SERVICES \nIGF::CL::IGF","EASTERN RESEARCH GROUP, INC.",EPC12028,NaN,112947395,"$113,846.00",2,2
2,AB92,R&D- COMMUNITY SERVICE/DEVELOPMENT: OTHER (APPLIED RESEARCH/EXPLORATORY DEVELOPMENT),ENVIRONMENTAL PROTECTION AGENCY,ENVIRONMENTAL PROTECTION AGENCY,ENVIRONMENTAL PROTECTION AGENCY,NaN,NaN,NaN,3/3/2015,FULL AND OPEN COMPETITION,NaN,COST PLUS FIXED FEE,"CLOSELY ASSOCIATED - ENVIRONMENTAL INDICATORS AND REPORT ON THE ENVIRONMENT PROGRAM (ROE) SCIENTIFIC, TECHNICAL, AND ADMINISTRATIVE SUPPORT SERVICES \nIGF::CL::IGF","EASTERN RESEARCH GROUP, INC.",EPC12028,NaN,112947395,"$62,055.00",3,3
3,AB92,R&D- COMMUNITY SERVICE/DEVELOPMENT: OTHER (APPLIED RESEARCH/EXPLORATORY DEVELOPMENT),ENVIRONMENTAL PROTECTION AGENCY,ENVIRONMENTAL PROTECTION AGENCY,ENVIRONMENTAL PROTECTION AGENCY,NaN,NaN,NaN,4/7/2015,FULL AND OPEN COMPETITION,NaN,COST PLUS FIXED FEE,"CLOSELY ASSOCIATED - ENVIRONMENTAL INDICATORS AND REPORT ON THE ENVIRONMENT PROGRAM (ROE) SCIENTIFIC, TECHNICAL, AND ADMINISTRATIVE SUPPORT SERVICES \nIGF::CL::IGF","EASTERN RESEARCH GROUP, INC.",EPC12028,NaN,112947395,"$58,714.00",4,4
4,AB92,R&D- COMMUNITY SERVICE/DEVELOPMENT: OTHER (APPLIED RESEARCH/EXPLORATORY DEVELOPMENT),ENVIRONMENTAL PROTECTION AGENCY,ENVIRONMENTAL PROTECTION AGENCY,ENVIRONMENTAL PROTECTION AGENCY,NaN,NaN,NaN,4/30/2015,FULL AND OPEN COMPETITION,NaN,COST PLUS FIXED FEE,"CLOSELY ASSOCIATED - ENVIRONMENTAL INDICATORS AND REPORT ON THE ENVIRONMENT PROGRAM (ROE) SCIENTIFIC, TECHNICAL, AND ADMINISTRATIVE SUPPORT SERVICES \nIGF::CL::IGF","EASTERN RESEARCH GROUP, INC.",EPC12028,NaN,112947395,"$150,000.00",5,5


In [561]:
# Lets join df and Topic_df

Result = pd.merge(df,
                 Topic_df,
                 on='Doc_ID', 
                 how='left')

Result.head(10)

,Product or Service Code,Product or Service Description,Contracting Agency Name,Contracting Department Name,Funding Agency Name,Principal Place of Performance City Name,Principal Place of Performance State Code,Principal Place of Performance Country Name,Signed Date,Extent Competed,...,Type of Contract,Description of Requirement,Vendor Name,PIID,Referenced IDV PIID,DUNS Number,Action Obligation,Doc_ID,Row_Num,Topic_Number
0,AB92,R&D- COMMUNITY SERVICE/DEVELOPMENT: OTHER (APPLIED RESEARCH/EXPLORATORY DEVELOPMENT),ENVIRONMENTAL PROTECTION AGENCY,ENVIRONMENTAL PROTECTION AGENCY,ENVIRONMENTAL PROTECTION AGENCY,NaN,NaN,NaN,11/3/2014,FULL AND OPEN COMPETITION,...,COST PLUS FIXED FEE,"CLOSELY ASSOCIATED - ENVIRONMENTAL INDICATORS AND REPORT ON THE ENVIRONMENT PROGRAM (ROE) SCIENTIFIC, TECHNICAL, AND ADMINISTRATIVE SUPPORT SERVICES \nIGF::CL::IGF","EASTERN RESEARCH GROUP, INC.",EPC12028,NaN,112947395,"$131,940.00",1,1,0.0
1,AB92,R&D- COMMUNITY SERVICE/DEVELOPMENT: OTHER (APPLIED RESEARCH/EXPLORATORY DEVELOPMENT),ENVIRONMENTAL PROTECTION AGENCY,ENVIRONMENTAL PROTECTION AGENCY,ENVIRONMENTAL PROTECTION AGENCY,NaN,NaN,NaN,12/24/2014,FULL AND OPEN COMPETITION,...,COST PLUS FIXED FEE,"CLOSELY ASSOCIATED - ENVIRONMENTAL INDICATORS AND REPORT ON THE ENVIRONMENT PROGRAM (ROE) SCIENTIFIC, TECHNICAL, AND ADMINISTRATIVE SUPPORT SERVICES \nIGF::CL::IGF","EASTERN RESEARCH GROUP, INC.",EPC12028,NaN,112947395,"$113,846.00",2,2,0.0
2,AB92,R&D- COMMUNITY SERVICE/DEVELOPMENT: OTHER (APPLIED RESEARCH/EXPLORATORY DEVELOPMENT),ENVIRONMENTAL PROTECTION AGENCY,ENVIRONMENTAL PROTECTION AGENCY,ENVIRONMENTAL PROTECTION AGENCY,NaN,NaN,NaN,3/3/2015,FULL AND OPEN COMPETITION,...,COST PLUS FIXED FEE,"CLOSELY ASSOCIATED - ENVIRONMENTAL INDICATORS AND REPORT ON THE ENVIRONMENT PROGRAM (ROE) SCIENTIFIC, TECHNICAL, AND ADMINISTRATIVE SUPPORT SERVICES \nIGF::CL::IGF","EASTERN RESEARCH GROUP, INC.",EPC12028,NaN,112947395,"$62,055.00",3,3,0.0
3,AB92,R&D- COMMUNITY SERVICE/DEVELOPMENT: OTHER (APPLIED RESEARCH/EXPLORATORY DEVELOPMENT),ENVIRONMENTAL PROTECTION AGENCY,ENVIRONMENTAL PROTECTION AGENCY,ENVIRONMENTAL PROTECTION AGENCY,NaN,NaN,NaN,4/7/2015,FULL AND OPEN COMPETITION,...,COST PLUS FIXED FEE,"CLOSELY ASSOCIATED - ENVIRONMENTAL INDICATORS AND REPORT ON THE ENVIRONMENT PROGRAM (ROE) SCIENTIFIC, TECHNICAL, AND ADMINISTRATIVE SUPPORT SERVICES \nIGF::CL::IGF","EASTERN RESEARCH GROUP, INC.",EPC12028,NaN,112947395,"$58,714.00",4,4,0.0
4,AB92,R&D- COMMUNITY SERVICE/DEVELOPMENT: OTHER (APPLIED RESEARCH/EXPLORATORY DEVELOPMENT),ENVIRONMENTAL PROTECTION AGENCY,ENVIRONMENTAL PROTECTION AGENCY,ENVIRONMENTAL PROTECTION AGENCY,NaN,NaN,NaN,4/30/2015,FULL AND OPEN COMPETITION,...,COST PLUS FIXED FEE,"CLOSELY ASSOCIATED - ENVIRONMENTAL INDICATORS AND REPORT ON THE ENVIRONMENT PROGRAM (ROE) SCIENTIFIC, TECHNICAL, AND ADMINISTRATIVE SUPPORT SERVICES \nIGF::CL::IGF","EASTERN RESEARCH GROUP, INC.",EPC12028,NaN,112947395,"$150,000.00",5,5,0.0
5,AB94,R&D- COMMUNITY SERVICE/DEVELOPMENT: OTHER (ENGINEERING DEVELOPMENT),ENVIRONMENTAL PROTECTION AGENCY,ENVIRONMENTAL PROTECTION AGENCY,ENVIRONMENTAL PROTECTION AGENCY,POTTSTOWN,PA,UNITED STATES,12/16/2014,NOT COMPETED UNDER SAP,...,FIRM FIXED PRICE,IGF::OT::IGF\nURBAN SOIL TAXONOMIC DESCRIPTIONS AND CHARACTERIZATION OF SOIL PROPERTIES APPLIED TO THE SUSTAINABLE USE OF VACANT LAND,"CEDARVILLE ENGINEERING GROUP, LLC",EP15C000028,NaN,962659947,"$108,750.00",6,6,NaN
6,AB94,R&D- COMMUNITY SERVICE/DEVELOPMENT: OTHER (ENGINEERING DEVELOPMENT),ENVIRONMENTAL PROTECTION AGENCY,ENVIRONMENTAL PROTECTION AGENCY,ENVIRONMENTAL PROTECTION AGENCY,POTTSTOWN,PA,UNITED STATES,5/8/2015,NOT COMPETED UNDER SAP,...,FIRM FIXED PRICE,IGF::OT::IGF\nCONTRACTOR TO PROVIDE GEOPROBE SERVICES THAT ARE NOT ABLE TO BE PROVIDED BY EPA IN SUPPORT OF URBAN SOIL TAXONOMIC DESCRIPTIONS AND CHARACTERIZATION OF SOIL PROPERTIES APPLIED TO THE SUSTAINABLE USE OF VACANT LAND,"CEDARVILLE ENGINEERING GROUP, LLC",EP15C000028,NaN,962659947,"$30,000.00",7,7,0.0
7,AB96,R&D- COMMUNITY SERVICE/DEVE